In [1]:
import cv2
import numpy as np

In [6]:
def calculate_white_pixels(image):
    """ 计算图像中白色像素的数量 """
    return np.sum(image == 255)

def get_center_of_mass(image):
    """ 计算图像的重心坐标 """
    white_pixels = np.argwhere(image == 255)
    if len(white_pixels) == 0:
        return image.shape[1] // 2, image.shape[0] // 2  # 返回图像中心
    center_y, center_x = np.mean(white_pixels, axis=0).astype(int)
    return center_x, center_y

def resize_and_center_image(template, student, output_size=(400, 400)):
    # Step 1: 计算两个图像中的白色像素数量
    S_T = calculate_white_pixels(template)
    S_C = calculate_white_pixels(student)

    print(f"Template white pixels: {S_T}")
    print(f"Student white pixels: {S_C}")
    
    # Step 2: 提取模板和临摹字的ROI区域
    x_T, y_T, w_T, h_T = cv2.boundingRect(template)
    template_roi = template[y_T:y_T+h_T, x_T:x_T+w_T]
    
    x_C, y_C, w_C, h_C = cv2.boundingRect(student)
    student_roi = student[y_C:y_C+h_C, x_C:x_C+w_C]
    
    # Step 3: 计算缩放因子并调整临摹字的大小
    scaling_factor = np.sqrt(S_T / S_C)
    new_width = int(w_C * scaling_factor)
    new_height = int(h_C * scaling_factor)
    resized_student = cv2.resize(student_roi, (new_width, new_height))
    
    # Step 4: 创建400x400的空白背景
    blank_template = np.ones(output_size, dtype=np.uint8) * 255  # 模板字空白图
    blank_student = np.ones(output_size, dtype=np.uint8) * 255   # 临摹字空白图
    
    # Step 5: 获取两个图像的重心
    template_center_x, template_center_y = get_center_of_mass(template_roi)
    student_center_x, student_center_y = get_center_of_mass(resized_student)
    
    # Step 6: 将模板字放到空白图的中心
    start_x_T = output_size[1] // 2 - w_T // 2
    start_y_T = output_size[0] // 2 - h_T // 2
    blank_template[start_y_T:start_y_T+h_T, start_x_T:start_x_T+w_T] = template_roi
    
    # Step 7: 将调整后的临摹字放到空白图的中心
    start_x_C = output_size[1] // 2 - resized_student.shape[1] // 2
    start_y_C = output_size[0] // 2 - resized_student.shape[0] // 2
    blank_student[start_y_C:start_y_C+resized_student.shape[0], start_x_C:start_x_C+resized_student.shape[1]] = resized_student
    
    # Step 8: 确保两个图像的白色像素数量相等
    new_S_T = calculate_white_pixels(blank_template)
    new_S_C = calculate_white_pixels(blank_student)
    
    print(f"New template white pixels: {new_S_T}")
    print(f"New student white pixels: {new_S_C}")

    if new_S_T != new_S_C:
        print("Warning: White pixel counts do not match after resizing.")
    else:
        print("White pixel counts match after resizing.")
    
    return blank_template, blank_student

# 调用此函数以处理图像并输出白色像素信息
def main():
    template_path = r"E:\python files\PycharmProjects\CompleteProject\thinned_image.png"
    student_path = r"E:\python files\PycharmProjects\CompleteProject\thinned_image1.png"

    # 读取灰度图像
    template = cv2.imread(template_path, cv2.IMREAD_GRAYSCALE)
    student = cv2.imread(student_path, cv2.IMREAD_GRAYSCALE)

    if template is None or student is None:
        print("Error loading images.")
        return
    
    # 调整临摹字大小并将它们都显示在400x400的空白图中
    aligned_template, aligned_student = resize_and_center_image(template, student)
    
    # 显示归一化后的结果
    cv2.imshow("Aligned Template", aligned_template)
    cv2.imshow("Aligned Student", aligned_student)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

if __name__ == "__main__":
    main()


Template white pixels: 806
Student white pixels: 812
New template white pixels: 91344
New student white pixels: 88589


In [3]:
# Zhang Suen 细化算法，进行骨架提取
def Zhang_Suen_thinning(img):
    # Get image shape
    H, W = img.shape

    # Prepare output image
    out = np.zeros((H, W), dtype=np.int32)
    out[img[..., 0] > 0] = 1

    # Inverse image (white background, black character)
    out = 1 - out

    while True:
        s1 = []
        s2 = []

        # Step 1 (raster scan)
        for y in range(1, H - 1):
            for x in range(1, W - 1):

                # Skip non-edge pixels
                if out[y, x] > 0:
                    continue

                # Condition 2
                f1 = 0
                if (out[y - 1, x + 1] - out[y - 1, x]) == 1: f1 += 1
                if (out[y, x + 1] - out[y - 1, x + 1]) == 1: f1 += 1
                if (out[y + 1, x + 1] - out[y, x + 1]) == 1: f1 += 1
                if (out[y + 1, x] - out[y + 1, x + 1]) == 1: f1 += 1
                if (out[y + 1, x - 1] - out[y + 1, x]) == 1: f1 += 1
                if (out[y, x - 1] - out[y + 1, x - 1]) == 1: f1 += 1
                if (out[y - 1, x - 1] - out[y, x - 1]) == 1: f1 += 1
                if (out[y - 1, x] - out[y - 1, x - 1]) == 1: f1 += 1

                if f1 != 1:
                    continue

                # Condition 3
                f2 = np.sum(out[y - 1:y + 2, x - 1:x + 2])
                if f2 < 2 or f2 > 6:
                    continue

                # Condition 4 and 5
                if (out[y - 1, x] + out[y, x + 1] + out[y + 1, x]) < 1: continue
                if (out[y, x + 1] + out[y + 1, x] + out[y, x - 1]) < 1: continue

                s1.append([y, x])

        for v in s1:
            out[v[0], v[1]] = 1

        # Step 2 (raster scan)
        for y in range(1, H - 1):
            for x in range(1, W - 1):

                if out[y, x] > 0:
                    continue

                # Same conditions as step 1
                f1 = 0
                if (out[y - 1, x + 1] - out[y - 1, x]) == 1: f1 += 1
                if (out[y, x + 1] - out[y - 1, x + 1]) == 1: f1 += 1
                if (out[y + 1, x + 1] - out[y, x + 1]) == 1: f1 += 1
                if (out[y + 1, x] - out[y + 1, x + 1]) == 1: f1 += 1
                if (out[y + 1, x - 1] - out[y + 1, x]) == 1: f1 += 1
                if (out[y, x - 1] - out[y + 1, x - 1]) == 1: f1 += 1
                if (out[y - 1, x - 1] - out[y, x - 1]) == 1: f1 += 1
                if (out[y - 1, x] - out[y - 1, x - 1]) == 1: f1 += 1

                if f1 != 1:
                    continue

                f2 = np.sum(out[y - 1:y + 2, x - 1:x + 2])
                if f2 < 2 or f2 > 6:
                    continue

                if (out[y - 1, x] + out[y, x + 1] + out[y, x - 1]) < 1: continue
                if (out[y - 1, x] + out[y + 1, x] + out[y, x - 1]) < 1: continue

                s2.append([y, x])

        for v in s2:
            out[v[0], v[1]] = 1

        if len(s1) < 1 and len(s2) < 1:
            break

    out = apply_template_removal(out, H, W)

    out = 1 - out
    out = out.astype(np.uint8) * 255

    return out

In [4]:
# 空洞或噪声消除模板
def apply_template_removal(out, H, W):
    for y in range(1, H-1):
        for x in range(1, W-1):
            P = out[y-1:y+2, x-1:x+2].flatten()
            if (P[1] * P[7] == 1 and sum([P[3], P[4], P[5], P[8]]) == 0) or \
               (P[5] * P[7] == 1 and sum([P[1], P[2], P[3], P[6]]) == 0) or \
               (P[1] * P[3] == 1 and sum([P[2], P[5], P[6], P[7]]) == 0) or \
               (P[3] * P[5] == 1 and sum([P[1], P[4], P[7], P[8]]) == 0) or \
               (sum([P[2], P[4], P[6], P[8]]) == 0 and sum([P[1], P[3], P[5], P[7]]) == 3):
                out[y, x] = 1
    return out

def apply_template(image, template):
    """根据给定模板进行处理，消除噪声或孔洞"""
    h, w = image.shape
    processed_image = image.copy()
    for i in range(1, h - 1):
        for j in range(1, w - 1):
            # 获取8邻域
            neighborhood = image[i - 1:i + 2, j - 1:j + 2]
            # 与模板匹配
            if np.array_equal(neighborhood, template):
                processed_image[i, j] = 0  # 将符合条件的前景点更改为背景点
    return processed_image

# 凹凸点和孤立点模板
templates_bumps_isolated = [
    np.array([[0, 0, 0], [0, 1, 0], [1, 1, 1]]),
    np.array([[1, 0, 0], [1, 1, 0], [1, 0, 0]]),

]
# 孔洞消除模板
templates_holes = [
    np.array([[1, 1, 1], [1, 0, 1], [1, 1, 1]]),

]

# 应用模板处理图像
def preprocess_image(image):
    # 处理凹凸点和孤立点
    for template in templates_bumps_isolated:
        image = apply_template(image, template)

    # 处理孔洞
    for template in templates_holes:
        image = apply_template(image, template)

    return image

def preprocess_images(image1, image2):
    """
    处理两个输入图像，分别处理凹凸点、孤立点和孔洞。
    """
    # 分别处理 image1 和 image2
    processed_image1 = preprocess_image(image1)
    processed_image2 = preprocess_image(image2)

    return processed_image1, processed_image2

In [5]:
#骨架相似度评价，用的是像素计算。（也可用坐标计算）
# 计算两个点之间的欧几里得距离
def euclidean_distance(p1, p2):
    return np.sqrt((p1[0] - p2[0]) ** 2 + (p1[1] - p2[1]) ** 2)


# 计算一个点到一组骨架点的最小距离
def point_to_skeleton_distance(point, skeleton_points):
    return min(euclidean_distance(point, sk_point) for sk_point in skeleton_points)


# 计算两个骨架图像之间的总距离
def calculate_total_distance(skeleton1, skeleton2):
    skeleton1_points = np.argwhere(skeleton1 == 255)  # 骨架为白色
    skeleton2_points = np.argwhere(skeleton2 == 255)  # 骨架为白色
    total_distance = sum(point_to_skeleton_distance(point, skeleton2_points) for point in skeleton1_points)
    return total_distance

In [6]:
# -----------------------------笔画提取--------------------------------
def extract_grid_image(imageData, position):
    height, width = imageData.shape[:2]
    grid_height = height // 3
    grid_width = width // 3
    x, y = position
    start_x = x * grid_width
    start_y = y * grid_height
    end_x = start_x + grid_width
    end_y = start_y + grid_height
    end_x = min(end_x, width)
    end_y = min(end_y, height)
    grid_image = imageData[start_y:end_y, start_x:end_x]
    return grid_image

# 定义函数计算图像的Hu矩
def calculate_hu_moments(image):
    if cv2.countNonZero(image) == 0:
        return np.zeros(7)  # 返回零矩阵避免 nan
    moments = cv2.moments(image)
    hu_moments = cv2.HuMoments(moments).flatten()
    return hu_moments


In [7]:
# 计算笔画相似度
def StrokeExtractionAndSimilarityEvaluation(imageData, reference_image):
    handwriting_hu_moments = calculate_hu_moments(imageData)
    template_hu_moments = calculate_hu_moments(reference_image)

    grid_positions = [(0, 0), (0, 1), (0, 2),
                      (1, 0), (1, 1), (1, 2),
                      (2, 0), (2, 1), (2, 2)]

    grid_weights = [0.1, 0.1, 0.1,
                    0.1, 0.5, 0.1,
                    0.1, 0.1, 0.1]

    handwriting_hu_moments_list = []
    template_hu_moments_list = []

    for position in grid_positions:
        handwriting_grid_image = extract_grid_image(imageData, position)
        template_grid_image = extract_grid_image(reference_image, position)

        handwriting_hu_moments = calculate_hu_moments(handwriting_grid_image)
        template_hu_moments = calculate_hu_moments(template_grid_image)

        handwriting_hu_moments_list.append(handwriting_hu_moments)
        template_hu_moments_list.append(template_hu_moments)

    correlation_coefficients = []
    for handwriting_hu_moments, template_hu_moments in zip(handwriting_hu_moments_list, template_hu_moments_list):
        correlation_coefficient = np.corrcoef(handwriting_hu_moments, template_hu_moments)[0, 1]
        if np.isnan(correlation_coefficient):
            correlation_coefficient = 0  # 避免 nan
        correlation_coefficients.append(correlation_coefficient)

    # 确保 correlation_coefficients 和 grid_weights 都是数值类型列表
    correlation_coefficients = np.array(correlation_coefficients, dtype=float)
    grid_weights = np.array(grid_weights, dtype=float)

    # 计算加权和
    weighted_sum = np.dot(correlation_coefficients, grid_weights)

    # 确保 weighted_sum 是浮点数并乘以常量
    weighted_sum = float(weighted_sum) * 76.924
    weighted_sum = round(weighted_sum, 2)

    # 根据相似度值返回相应的描述
    if weighted_sum < 20:
        description = '笔画极其不相似，与模板差异极大'
    elif weighted_sum < 40:
        description = '笔画很不相似，需要大幅度改进'
    elif weighted_sum < 60:
        description = '笔画不太相似，部分偏离模板'
    elif weighted_sum < 80:
        description = '笔画位置基本相似，但仍有改进空间'
    else:
        description = '笔画位置非常相似，与模板高度一致'

    print('笔画相似度:', weighted_sum)
    print(description)
    return weighted_sum, description

In [8]:
def main():
    template_path = r"E:\python files\PycharmProjects\CompleteProject\4.png"
    handwriting_path = r"E:\python files\PycharmProjects\CompleteProject\18.png"
    
    # 读取灰度图像
    template = cv2.imread(template_path, cv2.IMREAD_GRAYSCALE)
    handwriting = cv2.imread(handwriting_path, cv2.IMREAD_GRAYSCALE)
    
    if template is None or handwriting is None:
        print("Error loading images.")
        return
    
    # 调整临摹字大小并将它们都显示在400x400的空白图中
    aligned_template, aligned_handwriting = resize_and_center_image(template, handwriting)
    
    # 反色处理
    img1 = cv2.bitwise_not(aligned_template)
    img2 = cv2.bitwise_not(aligned_handwriting)
    
    # 骨架相似度计算
    out1 = Zhang_Suen_thinning(img1)
    out2 = Zhang_Suen_thinning(img2)
    aligned_template1, aligned_handwriting1 = preprocess_images(out1, out2)
 
    # print("Image Shape:", aligned_template1.shape)        #Image Shape: (400, 400)
    # print("Thinned Image Shape:", aligned_handwriting1.shape)          #Thinned Image Shape: (400, 400)
    
    # 计算骨架相似度
    skeleton_similarity_score = calculate_total_distance(aligned_template1, aligned_handwriting1)
    print("Skeleton Similarity Score:", skeleton_similarity_score)
    
    # 计算布局相似度，hu矩九宫格
    weighted_score, description = StrokeExtractionAndSimilarityEvaluation(aligned_template1, aligned_handwriting1)

    # 计算总分
    full_score = skeleton_similarity_score * 0.5 + weighted_score * 0.5
    print("总分：", full_score)
    print(description)

if __name__ == "__main__":
    main()

Skeleton Similarity Score: 0
笔画相似度: 0.0
笔画极其不相似，与模板差异极大
总分： 0.0
笔画极其不相似，与模板差异极大


E:\python files\PycharmProjects\CompleteProject\venv\Lib\site-packages\numpy\lib\function_base.py:2897: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
